In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from ast import literal_eval 
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
import warnings; warnings.simplefilter('ignore')


In [2]:
df = pd.read_csv('IMDb movies.csv')
dfn = pd.read_csv('IMDb names.csv')
dfr = pd.read_csv('IMDb ratings.csv')
dft = pd.read_csv('IMDb title_principals.csv')


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85855 entries, 0 to 85854
Data columns (total 22 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   imdb_title_id          85855 non-null  object 
 1   title                  85855 non-null  object 
 2   original_title         85855 non-null  object 
 3   year                   85855 non-null  object 
 4   date_published         85855 non-null  object 
 5   genre                  85855 non-null  object 
 6   duration               85855 non-null  int64  
 7   country                85791 non-null  object 
 8   language               85022 non-null  object 
 9   director               85768 non-null  object 
 10  writer                 84283 non-null  object 
 11  production_company     81400 non-null  object 
 12  actors                 85786 non-null  object 
 13  description            83740 non-null  object 
 14  avg_vote               85855 non-null  float64
 15  vo

In [5]:
dfn.head()

,imdb_name_id,name,birth_name,height,bio,birth_details,date_of_birth,place_of_birth,death_details,date_of_death,place_of_death,reason_of_death,spouses_string,spouses,divorces,spouses_with_children,children
0,nm0000001,Fred Astaire,Frederic Austerlitz Jr.,177.0,"Fred Astaire was born in Omaha, Nebraska, to J...","May 10, 1899 in Omaha, Nebraska, USA",1899-05-10,"Omaha, Nebraska, USA","June 22, 1987 in Los Angeles, California, USA ...",1987-06-22,"Los Angeles, California, USA",pneumonia,Robyn Smith (27 June 1980 - 22 June 1987) (hi...,2,0,1,2
1,nm0000002,Lauren Bacall,Betty Joan Perske,174.0,Lauren Bacall was born Betty Joan Perske on Se...,"September 16, 1924 in The Bronx, New York City...",1924-09-16,"The Bronx, New York City, New York, USA","August 12, 2014 in New York City, New York, US...",2014-08-12,"New York City, New York, USA",stroke,Jason Robards (4 July 1961 - 10 September 196...,2,1,2,3
2,nm0000003,Brigitte Bardot,Brigitte Bardot,166.0,"Brigitte Bardot was born on September 28, 1934...","September 28, 1934 in Paris, France",1934-09-28,"Paris, France",NaN,NaN,NaN,NaN,Bernard d'Ormale (16 August 1992 - present)\n...,4,3,1,1
3,nm0000004,John Belushi,John Adam Belushi,170.0,"John Belushi was born in Chicago, Illinois, US...","January 24, 1949 in Chicago, Illinois, USA",1949-01-24,"Chicago, Illinois, USA","March 5, 1982 in Hollywood, Los Angeles, Calif...",1982-03-05,"Hollywood, Los Angeles, California, USA",acute cocaine and heroin intoxication,Judith Belushi-Pisano (31 December 1976 - 5 M...,1,0,0,0
4,nm0000005,Ingmar Bergman,Ernst Ingmar Bergman,179.0,"Ernst Ingmar Bergman was born July 14, 1918, t...","July 14, 1918 in Uppsala, Uppsala län, Sweden",1918-07-14,"Uppsala, Uppsala län, Sweden","July 30, 2007 in Fårö, Gotlands län, Sweden (...",2007-07-30,"Fårö, Gotlands län, Sweden",natural causes,Ingrid Bergman (11 November 1971 - 20 May 199...,5,4,5,8


In [6]:
df.isnull().sum() # verificando se tinha algum valor NaN no avg_test ou votes


imdb_title_id                0
title                        0
original_title               0
year                         0
date_published               0
genre                        0
duration                     0
country                     64
language                   833
director                    87
writer                    1572
production_company        4455
actors                      69
description               2115
avg_vote                     0
votes                        0
budget                   62145
usa_gross_income         70529
worlwide_gross_income    54839
metascore                72550
reviews_from_users        7597
reviews_from_critics     11797
dtype: int64

In [7]:
colunas = ['original_title', 'year', 'genre','country','director','actors', 'avg_vote','votes']
dfnovo = df[colunas]


In [8]:
dfnovo.head()

,original_title,year,genre,country,director,actors,avg_vote,votes
0,Miss Jerry,1894,Romance,USA,Alexander Black,"Blanche Bayliss, William Courtenay, Chauncey D...",5.9,154
1,The Story of the Kelly Gang,1906,"Biography, Crime, Drama",Australia,Charles Tait,"Elizabeth Tait, John Tait, Norman Campbell, Be...",6.1,589
2,Den sorte drøm,1911,Drama,"Germany, Denmark",Urban Gad,"Asta Nielsen, Valdemar Psilander, Gunnar Helse...",5.8,188
3,Cleopatra,1912,"Drama, History",USA,Charles L. Gaskill,"Helen Gardner, Pearl Sindelar, Miss Fielding, ...",5.2,446
4,L'Inferno,1911,"Adventure, Drama, Fantasy",Italy,"Francesco Bertolini, Adolfo Padovan","Salvatore Papa, Arturo Pirovano, Giuseppe de L...",7.0,2237


In [9]:
votes = dfnovo[dfnovo["votes"].notnull()]["votes"].astype('int')

In [10]:
dfnovo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85855 entries, 0 to 85854
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   original_title  85855 non-null  object 
 1   year            85855 non-null  object 
 2   genre           85855 non-null  object 
 3   country         85791 non-null  object 
 4   director        85768 non-null  object 
 5   actors          85786 non-null  object 
 6   avg_vote        85855 non-null  float64
 7   votes           85855 non-null  int64  
dtypes: float64(1), int64(1), object(6)
memory usage: 5.2+ MB


In [11]:
avg_vote = dfnovo[dfnovo["avg_vote"].notnull()]["avg_vote"].astype('float')

In [12]:
dfnovo.director.fillna("Nenhum diretor", inplace = True)
dfnovo.actors.fillna(" Nenhum Ator", inplace = True)

In [13]:
C = avg_vote.mean()
C

5.898655873274613

In [22]:
#calculando o quartil para entrar no top filmes
m = votes.quantile(0.40)
m

335.0

In [23]:
 qualified = dfnovo[(dfnovo['votes'] >= m) & (dfnovo['votes'].notnull()) & (dfnovo['avg_vote'].notnull())]

In [24]:
qualified.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51562 entries, 1 to 85851
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   original_title  51562 non-null  object 
 1   year            51562 non-null  object 
 2   genre           51562 non-null  object 
 3   country         51545 non-null  object 
 4   director        51562 non-null  object 
 5   actors          51562 non-null  object 
 6   avg_vote        51562 non-null  float64
 7   votes           51562 non-null  int64  
dtypes: float64(1), int64(1), object(6)
memory usage: 3.5+ MB


In [25]:
qualified.shape

(51562, 8)

In [22]:
def imdb_qualified(x):
    v = x["votes"]
    R = x["avg_vote"]
    return (v/(v+m) * R) + (m/(m+v) * C)

In [23]:
qualified['wr'] = qualified.apply(imdb_qualified, axis = 1)

In [24]:
qualified.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4293 entries, 165 to 85504
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   original_title  4293 non-null   object 
 1   year            4293 non-null   object 
 2   genre           4293 non-null   object 
 3   country         4293 non-null   object 
 4   director        4293 non-null   object 
 5   actors          4293 non-null   object 
 6   avg_vote        4293 non-null   float64
 7   votes           4293 non-null   int64  
 8   wr              4293 non-null   float64
dtypes: float64(2), int64(1), object(6)
memory usage: 335.4+ KB


In [25]:
qualified = qualified.sort_values("wr", ascending = False).head(250)

In [26]:
qualified.head(20)

,original_title,year,genre,country,director,actors,avg_vote,votes,wr
28453,The Shawshank Redemption,1994,Drama,USA,Frank Darabont,"Tim Robbins, Morgan Freeman, Bob Gunton, Willi...",9.3,2278845,9.250845
15528,The Godfather,1972,"Crime, Drama",USA,Francis Ford Coppola,"Marlon Brando, Al Pacino, James Caan, Richard ...",9.2,1572674,9.131312
48078,The Dark Knight,2008,"Action, Crime, Drama","USA, UK",Christopher Nolan,"Christian Bale, Heath Ledger, Aaron Eckhart, M...",9.0,2241615,8.954447
16556,The Godfather: Part II,1974,"Crime, Drama",USA,Francis Ford Coppola,"Al Pacino, Robert Duvall, Diane Keaton, Robert...",9.0,1098714,8.908460
28381,Pulp Fiction,1994,"Crime, Drama",USA,Quentin Tarantino,"Tim Roth, Amanda Plummer, Laura Lovelace, John...",8.9,1780147,8.844698
34127,The Lord of the Rings: The Return of the King,2003,"Action, Adventure, Drama","New Zealand, USA",Peter Jackson,"Noel Appleby, Ali Astin, Sean Astin, David Ast...",8.9,1604280,8.838759
27629,Schindler's List,1993,"Biography, Drama, History",USA,Steven Spielberg,"Liam Neeson, Ben Kingsley, Ralph Fiennes, Caro...",8.9,1183248,8.817567
8973,12 Angry Men,1957,"Crime, Drama",USA,Sidney Lumet,"Martin Balsam, John Fiedler, Lee J. Cobb, E.G....",8.9,668473,8.757109
57475,Inception,2010,"Action, Adventure, Sci-Fi","USA, UK",Christopher Nolan,"Leonardo DiCaprio, Joseph Gordon-Levitt, Ellen...",8.8,2002816,8.752387
32487,Fight Club,1999,Drama,"USA, Germany",David Fincher,"Edward Norton, Brad Pitt, Meat Loaf, Zach Gren...",8.8,1807440,8.747333


In [28]:
#Sistema de recomendação 2



IndexError: Too many levels: Index has only 1 level, not 2

In [ ]:
#Função de top filmes por categoria 